In [ ]:
!pip install -q transformers

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/MT-Kh-Vi/test.csv", encoding="utf-8")

df.head()

,vi,km
0,Vượt qua những tác động tiêu cực do dịch COVID...,ដោយជំនះពុះពារលើផលប៉ះពាល់អវិជ្ជមាន ដោយសារជំងឺរា...
1,"Cùng với đó, các nước thành viên sẽ tăng cường...",ទន្ទឹមជាមួយគ្នានោះ បណ្ដាប្រទេសសមាជិកនឹងបង្កើនក...
2,Hợp tác Mekong-Lan Thương vì một khu vực Mekon...,កិច្ចសហប្រតិបត្តិការមេគង្គ-ឡានឆាងដើម្បីតំបន់មេ...
3,"Vì vậy, phục hồi kinh tế hậu Covid-19 cũng là ...",ដូច្នេះការស្ដារសេដ្ឋកិច្ចឡើងវិញក្រោយពីកូវីដ ១៩...
4,Điều này cho thấy sự hiện diện ngày càng mạnh ...,ប្រការនេះបង្ហាញពីវត្តមានកាន់តែខ្លាំងឡើងៗ របស់វ...


In [ ]:
src_txts = []
ref_txts = []

for _, row in df.iterrows():
  src_txts.append(row["vi"])
  ref_txts.append(row["km"])

# **Model**

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "sail/Sailor2-1B-Chat"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto")
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/checkpoint")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "left"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

# **Inference**

In [ ]:
def translate_sailor2_batch(texts_vi, batch_size=8):
    # prompts = [
    #     f"<|im_start|>user\nDịch câu tiếng Khmer sau sang tiếng Việt (không giải thích): {text}<|im_end|>\n<|im_start|>assistant\n"
    #     for text in texts_vi
    # ]
#     system_prompt = (
#     "Bạn là một chuyên gia dịch máy giữa tiếng Việt và tiếng Khmer. "
#     "Nhiệm vụ của bạn là chỉ đưa ra bản dịch chính xác mà không thêm bất kỳ giải thích, chú thích hay lặp lại nội dung đầu vào."
# )

#     translation_prompt = '''Hãy dịch câu sau từ tiếng Việt sang tiếng Khmer. Không giải thích, chỉ trả về bản dịch:

#     Ví dụ:
#     Input: Nó có nhiều màu, mỗi màu lại có một vị, mà ăn cả một đĩa to chỉ có hơn chục nghìn.
#     Output: វាមានពណ៌ជាច្រើន ពណ៌នីមួយៗមានរសជាតិឆ្ងាញ់ ប៉ុន្តែការញ៉ាំចានធំមានតែច្រើនជាងមួយម៉ឺនទេ។

#     Câu cần dịch:
#     Input: {}
#     Output:'''

#     system_prompt = (
#     "You are a professional translator specialized in Vietnamese–Khmer translations. "
#     "Translate accurately without explanations or repeating the input."
# )

#     translation_prompt = '''
#     ## Instruction:
#     Translate the following sentence from Vietnamese to Khmer. Do not explain or repeat the input. Just output the Khmer sentence.

#     ## Example:
#     ### Input:
#     Nó có nhiều màu, mỗi màu lại có một vị, mà ăn cả một đĩa to chỉ có hơn chục nghìn.
#     ### Output:
#     វាមានពណ៌ជាច្រើន ពណ៌នីមួយៗមានរសជាតិឆ្ងាញ់ ប៉ុន្តែការញ៉ាំចានធំមានតែច្រើនជាងមួយម៉ឺនទេ។

#     ## Task:
#     ### Input:
#     {}
#     ### Output:
#     '''

#     system_prompt = (
#     "You are a strict machine translation expert. Your only task is to translate from Vietnamese to Khmer. "
#     "You must return only the translation — no explanation, no repetition of the input, no formatting, and no additional comments."
# )

#     translation_prompt = '''
#     ## Instruction:
#     Translate the following sentence from Vietnamese to Khmer.
#     ⚠️ Strict rules:
#     - ❌ Do NOT repeat the input.
#     - ❌ Do NOT explain anything.
#     - ❌ Do NOT include 'Input:' or 'Output:' in your response.
#     - ✅ ONLY return the translated Khmer sentence.

#     ## Example:
#     Input: Nó có nhiều màu, mỗi màu lại có một vị, mà ăn cả một đĩa to chỉ có hơn chục nghìn.
#     Output: វាមានពណ៌ជាច្រើន ពណ៌នីមួយៗមានរសជាតិឆ្ងាញ់ ប៉ុន្តែការញ៉ាំចានធំមានតែច្រើនជាងមួយម៉ឺនទេ។

#     ## Task:
#     Input: {}
#     Output:
#     '''
    translation_prompt = '''Dịch câu sau từ tiếng Việt sang tiếng Khmer: {}'''




    # prompts = [
    #     f"<|im_start|>user\n{translation_prompt.format(text)}<|im_end|>\n<|im_start|>assistant\n" for text in texts_vi
    # ]
    texts = []
    for i in range(0, len(texts_vi)):
      messages = [
          # {"role":"system", "content": system_prompt},
          {"role": "user", "content": translation_prompt.format(texts_vi[i])}]
      texts.append(tokenizer.apply_chat_template(
          messages, tokenize=False, add_generation_prompt=True))

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256
        )

    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    results = [text.split("assistant\n")[-1].strip() for text in decoded_outputs]
    # results = [text.split("#Output (Vietnamese):")[-1].strip() for text in decoded_outputs]
    return results



In [ ]:
def batchify(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]


In [ ]:
predictions = []
batch_size = 16

for i, src in enumerate(batchify(src_txts, batch_size)):
  print(i)
  pred = translate_sailor2_batch(src)
  predictions.extend(pred)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [ ]:
for i, pred in enumerate(predictions):
  print(f"Source: {src_txts[i]}")
  print(f"Ref: {ref_txts[i]}")
  print(f"Predict: {pred}")
  print("------------------------------------")

Source: Vượt qua những tác động tiêu cực do dịch COVID - 19 gây ra, 9 tháng qua, Việt Nam thành công với mức tăng trưởng GDP 2,12%.
Ref: ដោយជំនះពុះពារលើផលប៉ះពាល់អវិជ្ជមាន ដោយសារជំងឺរាតត្បាតកូវីដ ១៩ នោះ ក្នុងរយៈពេល ៩ ខែកន្លងទៅ វៀតណាមទទួលបានជោគជ័យជាមួយនឹងកំណើន GDP ២,១២%។
Predict: ឆ្លងកាត់រាល់ឥទ្ធិពលអវិជ្ជមានដោយសារជំងឺរាតត្បាត Covid-19 នាអំឡុងចន្លោះ៩ខែកន្លងទៅ វៀតណាមទទួលបានគឺកំណើន GDP ២,១២%។
------------------------------------
Source: Cùng với đó, các nước thành viên sẽ tăng cường hợp tác y tế để ứng phó với đại dịch Covid-19; Bảo đảm sự ổn định cho hoạt động sản xuất và các chuỗi cung ứng khu vực, duy trì đà tăng trưởng kinh tế; Tiếp tục thúc đẩy phát triển bền vững, góp phần thực hiện thành công các mục tiêu SDGs 2030; Đẩy mạnh hợp tác trong lĩnh vực giáo dục đào tạo và nguồn lực chuyển đổi nguồn nhân lực cho khu vực Mekong.
Ref: ទន្ទឹមជាមួយគ្នានោះ បណ្ដាប្រទេសសមាជិកនឹងបង្កើនកិច្ចសហប្រតិបត្តិការសុខាភិបាលដើម្បីឆ្លើយតបនឹងជម្ងឺរាតត្បាតសកល Covid-19 ធានា ស្ថិរភាពសម្រាប់សកម្មភាពផលិតកម្មនិងខ្សែ

# **Evaluation**

In [ ]:
import pandas as pd

df = pd.DataFrame({
    "Source Text": src_txts,
    "Reference Text": ref_txts,
    "Prediction": predictions
})

df.to_csv("output.csv", index=False, encoding="utf-8-sig")

print("Đã lưu vào output.csv")

Đã lưu vào output.csv


In [ ]:
!pip install -q pyvi nltk khmer-nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.7 MB/s eta 0:00:00


In [ ]:
import nltk.translate.chrf_score
from nltk.translate.meteor_score import single_meteor_score
from khmernltk import word_tokenize
import nltk
import json
import pandas as pd

import nltk.translate.nist_score
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from pyvi import ViTokenizer

def tokenize_vi(sentence):
    return ViTokenizer.tokenize(sentence).split()

In [ ]:
def compute_nist(reference, translation):
    return nltk.translate.chrf_score.sentence_chrf(reference, translation)

def compute_meteor(reference, translation):
    # Use tokenized strings for METEOR score
    return single_meteor_score(reference, translation)

def compute_score(reference, translation):
    reference_tokens = word_tokenize(reference)
    translation_tokens = word_tokenize(translation)
    return compute_meteor(reference_tokens, translation_tokens), nltk.translate.bleu_score.sentence_bleu([reference], translation), compute_nist(reference_tokens, translation_tokens)

In [ ]:
bleu_score = 0
meteor_score = 0
bleun_score = 0
ter_score = 0
nist_score =0
id = 0
sub_scores = []
for prediction, reference in zip(predictions, ref_txts):
    m_score, bn_score, ni_score = compute_score(reference, prediction)
    # bleu_score += b_score
    meteor_score += m_score
    bleun_score += bn_score
    # ter_score += t_score
    nist_score += ni_score
    score = {
        'id': id,
        'bleu_score': bn_score,
        # 'bleu_score': b_score,
        'meteor_score': m_score,
        'chrF_score': ni_score
    }
    sub_scores.append(score)
    id += 1
result = {
    'bleu_score': bleun_score / (len(ref_txts)),
    # 'bleu_score': bleu_score / (len(prediction_data)),
    'meteor_score': meteor_score / (len(ref_txts)),
    'chrF_score': nist_score / (len(ref_txts))
}

| 2025-08-10 17:18:38,038 | INFO | khmer-nltk | Loaded model from /usr/local/lib/python3.11/dist-packages/khmernltk/word_tokenize/sklearn_crf_ner_10000.sav |
INFO:khmer-nltk:Loaded model from /usr/local/lib/python3.11/dist-packages/khmernltk/word_tokenize/sklearn_crf_ner_10000.sav


In [ ]:
result

{'bleu_score': 0.5233153860618197,
 'meteor_score': 0.47540563915565576,
 'chrF_score': 0.5172465528664355}

In [ ]:
!pip install -q evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

bleu_score = metric.compute(predictions=predictions, references=ref_txts)
bleu_score

{'score': 11.641587613292153,
 'counts': [4559, 2206, 1089, 499],
 'totals': [14565, 12565, 10598, 8797],
 'precisions': [31.301064194987983,
  17.556705133306803,
  10.275523683713908,
  5.672388314198022],
 'bp': 0.8702002461325524,
 'sys_len': 14565,
 'ref_len': 16590}